In [3]:
import csv
import numpy as np
from scipy.io import wavfile # get the api
from scipy.fftpack import fft
from scipy import signal

import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 
from scipy.fftpack import dct

import soundfile as sf

In [4]:


#sample_rate, data = wavfile.read(TRAIN_FOLDER+"92.wav")

#print(len(data),sample_rate)
#if data.ndim > 1 : # ignore  channels 2+
#   data = data[:, 0]
#


#nperseg = 2*(nFreqBin-1)
#noverlap = np.floor((SamplesPerPicture - nTimeBin*nperseg)/(1-nTimeBin))


#nfbin=(nperseg/2)+1


#nTbin = np.floor((len(data) - noverlap)/(nperseg -noverlap))
#print(len(data))
#noverlap = int(round(step_size * TARGETSampleRate / 1e3))
#freqs, times, spec = signal.spectrogram(data,fs=TARGETSampleRate,window='hann',nperseg=nperseg,noverlap=noverlap)#,scaling ='spectrum')
#log_specgram = np.log(spec.T.astype(np.float32) + eps)
#plt.figure(figsize=(10,10))
#plt.imshow(log_specgram.T)
#print(log_specgram.shape)
#plt.show()

In [5]:
#nperseg = int(round(window_size * TARGETSampleRate / 1e3))
#nperseg = int(round(duration*TARGETSampleRate))
#nperseg = int(round(len(data)/300))# for 0 overlap time bins = data length defided by window count


#noverlap = int(round(nperseg/8)) # Effects number of time bins (larger overlap increases time bins)

#freqs, times, spec = signal.spectrogram(audio_data,fs=TARGETSampleRate,window='hann')#,scaling ='spectrum')




In [6]:
TRAIN_PATH= './../data/urban-sound-classification/train/'
TEST_PATH= './../data/urban-sound-classification/test/'

TRAIN_CSV = TRAIN_PATH+'train.csv'
TRAIN_FOLDER = TRAIN_PATH+'Train/'

TEST_CSV = TEST_PATH+'test.csv'
TEST_FOLDER = TEST_PATH+'Test/'

In [7]:
#dimension=224
dimension=300
pictureTimeLength = 1
TargetSampleRate = 44100

In [9]:

SamplesPerPicture = pictureTimeLength * TargetSampleRate
nTimeBin =dimension # width
nFreqBin =dimension # height
eps=1e-10

nperseg =int(2*(nFreqBin-1))
noverlap = int(np.floor((SamplesPerPicture - nTimeBin*nperseg)/(1-nTimeBin)))
print (nperseg,noverlap)

598 452


In [37]:
OUTPUT_FOLDER_CLASSES = '../data/SciPy300/'

In [38]:
Labels=[]
Training_File_Dict={}
with open(TRAIN_CSV) as f:
    readCSV = csv.reader(f, delimiter=',')
    for row  in readCSV:
        if(not row[0]=='ID'):
            Training_File_Dict[row[0]]=row[1]
            if(not row[1] in Labels):
                Labels.append(row[1])

In [39]:
PathDictrionary={}
for i in Labels:
    LabeldDirectory = OUTPUT_FOLDER_CLASSES+"train/"+i+"/"
    PathDictrionary[i]=LabeldDirectory
    if not os.path.exists(LabeldDirectory):
        os.makedirs(LabeldDirectory)

In [40]:
Training_File_Dict

{'0': 'siren',
 '1': 'street_music',
 '2': 'drilling',
 '3': 'siren',
 '4': 'dog_bark',
 '6': 'children_playing',
 '10': 'street_music',
 '11': 'drilling',
 '12': 'gun_shot',
 '15': 'dog_bark',
 '17': 'engine_idling',
 '18': 'siren',
 '19': 'gun_shot',
 '20': 'engine_idling',
 '22': 'air_conditioner',
 '24': 'air_conditioner',
 '26': 'engine_idling',
 '27': 'siren',
 '32': 'children_playing',
 '33': 'jackhammer',
 '35': 'gun_shot',
 '36': 'siren',
 '37': 'jackhammer',
 '38': 'street_music',
 '40': 'jackhammer',
 '42': 'engine_idling',
 '43': 'siren',
 '44': 'children_playing',
 '45': 'jackhammer',
 '46': 'engine_idling',
 '47': 'air_conditioner',
 '48': 'car_horn',
 '49': 'drilling',
 '50': 'siren',
 '52': 'gun_shot',
 '54': 'children_playing',
 '56': 'children_playing',
 '57': 'drilling',
 '59': 'gun_shot',
 '60': 'siren',
 '61': 'car_horn',
 '63': 'drilling',
 '65': 'street_music',
 '66': 'jackhammer',
 '67': 'children_playing',
 '68': 'dog_bark',
 '71': 'dog_bark',
 '72': 'street_mu

In [55]:
### New Multithreaded

def log_mel_spec_tfm(fname, src_path, dst_path):
    global TotalCounter
    channels=[]
    sample_rate=0
    try:
        channels ,sample_rate = sf.read(src_path)
    except:
        print(fname+" Could not be computed!")
        return
    #print(channels.shape,fname)
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels)))).T
    if(channels.ndim==2):
        channels=channels.T
    print(channels.T.shape,fname)
    channelcounter=0
    for x in channels:
        channelcounter+=1
        #print(x.shape,fname)
        audio_data = signal.resample(x,int(round(len(x) *TargetSampleRate/sample_rate)))
        freqs, times, spec = signal.spectrogram(audio_data,fs=TargetSampleRate,window='hann',nperseg=nperseg,noverlap=noverlap)#,scaling ='spectrum')
        log_specgram = np.log(spec.T.astype(np.float32) + eps)
        width = log_specgram.shape[0]
        #print(log_specgram.shape)
        
        if width >= dimension:
            for i in range(0,int(np.floor(width/dimension))):
                out= np.zeros(shape=(dimension,dimension))
                if((i+1)*dimension<=width):
                    out = log_specgram[(i)*dimension:(i+1)*dimension,:].T
                    
                    im = plt.imshow(out)
                    colors = im.cmap(im.norm(out))
                    data = colors.astype(np.float64) / np.max(colors) # normalize the data to 0 - 1
                    data = 255 * data # Now scale by 255
                    img = data.astype(np.uint8)
                    out = img[:,:,0:3] 
                plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), out) 
                
#log_mel_spec_tfm("1615.wav", TRAIN_FOLDER+"/1615.wav","./")
#plt.show()
#log_mel_spec_tfm("0.wav", TRAIN_FOLDER+"/0.wav","./")
#plt.show()
#return;

counter = 0
filesToProcess=[]
for filename in os.listdir(TRAIN_FOLDER):
    if counter%100==0:
        print("Prepared thus far",counter)
    if filename.endswith(".wav") or filename.endswith(".m4a") or filename.endswith(".mp3"):
        filesToProcess.append(filename)
        counter +=1
    else:
        print("skipped: "+filename)
with multiprocessing.Pool(16) as p:
    outputDir = PathDictrionary[Training_File_Dict[filename.replace(".wav","")]]
    print(outputDir)
    p.starmap(log_mel_spec_tfm, [(f,TRAIN_FOLDER+'/'+str(f),PathDictrionary[Training_File_Dict[f.replace(".wav","")]]) for f in filesToProcess])
    print("Finished The Pool")
print("Done generating Spectra!")

Prepared thus far 0
Prepared thus far 100
Prepared thus far 200
Prepared thus far 300
Prepared thus far 400
Prepared thus far 500
Prepared thus far 600
Prepared thus far 700
Prepared thus far 800
skipped: .DS_Store
Prepared thus far 900
Prepared thus far 1000
Prepared thus far 1100
Prepared thus far 1200
Prepared thus far 1300
Prepared thus far 1400
Prepared thus far 1500
Prepared thus far 1600
Prepared thus far 1700
Prepared thus far 1800
skipped: 3121.png
Prepared thus far 1900
Prepared thus far 2000
Prepared thus far 2100
Prepared thus far 2200
Prepared thus far 2300
Prepared thus far 2400
Prepared thus far 2500
Prepared thus far 2600
Prepared thus far 2700
Prepared thus far 2800
Prepared thus far 2900
Prepared thus far 3000
Prepared thus far 3100
Prepared thus far 3200
Prepared thus far 3300
Prepared thus far 3400
Prepared thus far 3500
Prepared thus far 3600
Prepared thus far 3700
Prepared thus far 3800
Prepared thus far 3900
Prepared thus far 4000
Prepared thus far 4100
Prepared 

(176400, 2) 6934.wav
(176400, 2) 648.wav
(192000, 2) 2984.wav
(192000, 2) 2982.wav
(176400, 2) 2995.wav
(192000, 2) 8464.wav
(192000, 2) 677.wav
(176400, 2) 1134.wav
(176400, 2) 2604.wav
(384000, 2) 4896.wav
(192000, 2) 6138.wav
(192000, 2) 6077.wav
(176400, 2) 8077.wav
(176400, 2) 3859.wav
(176400, 2) 7975.wav
(176400, 2) 5021.wav
(176400, 2) 1083.wav
(109439, 2) 1244.wav
(192000, 2) 1326.wav
(192000, 2) 3442.wav
(163114, 2) 7792.wav
(176400, 2) 7369.wav
(192000, 2) 1096.wav
(176400, 2) 8711.wav
(176400, 2) 7226.wav
(29330, 2) 5353.wav
(44100, 1) 841.wav
(384000, 2) 6059.wav
(192000, 2) 5352.wav
(176400, 2) 8677.wav
(176400, 2) 2832.wav
(192000, 2) 699.wav
(176400, 2) 2996.wav
(176400, 2) 2577.wav
(176400, 2) 271.wav
(165600, 2) 528.wav
(176400, 2) 8512.wav
(176400, 2) 6449.wav
(33074, 1) 8673.wav
(176400, 2) 6886.wav
(192000, 2) 1916.wav
(96000, 2) 3644.wav
(192000, 2) 3871.wav
(176400, 2) 1736.wav
(176400, 2) 1652.wav
(176400, 2) 6882.wav
(176400, 2) 5191.wav
(192000, 1) 7636.wav
(1

(176400, 2) 3691.wav
(176400, 2) 2002.wav
(384000, 2) 3069.wav
(176400, 2) 5432.wav
(384000, 2) 1678.wav
(176400, 2) 114.wav
(192000, 2) 5594.wav
(192000, 2) 6459.wav
(171360, 2) 2004.wav
(176400, 2) 1647.wav
(108486, 2) 5768.wav
(176400, 2) 1640.wav
(176400, 2) 4473.wav
(176400, 2) 6931.wav
(176400, 2) 8114.wav
(176400, 2) 7383.wav
(192000, 2) 2371.wav
(176400, 2) 100.wav
(35280, 2) 4475.wav
(192000, 2) 4476.wav
(192000, 2) 6262.wav
(176400, 2) 8474.wav
(176400, 2) 1040.wav
(192000, 2) 8663.wav
(176400, 2) 8249.wav
(192000, 1) 512.wav
(176400, 2) 1719.wav
(97356, 2) 1898.wav
(192000, 2) 3319.wav
(192000, 2) 3861.wav
(79380, 2) 7227.wav
(176400, 1) 7431.wav
(192000, 2) 7197.wav
(176400, 2) 3283.wav
(192000, 2) 4714.wav
(67032, 2) 7550.wav
(8640, 2) 7747.wav
(192000, 2) 7800.wav
(67032, 2) 1656.wav
(192000, 2) 3657.wav
(176400, 2) 7231.wav
(176400, 2) 6896.wav
(192000, 2) 6270.wav
(51840, 2) 6139.wav
(36603, 1) 2210.wav
(176400, 2) 4661.wav
(176400, 2) 2439.wav
(176400, 2) 2016.wav
(176

(50235, 2) 941.wav
(384000, 2) 1133.wav
(192000, 2) 6869.wav
(192000, 2) 883.wav
(176400, 2) 5227.wav
(176400, 2) 6516.wav
(21609, 2) 1520.wav
(176400, 2) 7145.wav
(192000, 2) 3584.wav
(176400, 2) 5309.wav
(100948, 2) 7330.wav
(176400, 2) 7542.wav
(192000, 2) 288.wav
(50145, 2) 472.wav
(176400, 2) 3911.wav
(192000, 2) 1279.wav
(192000, 2) 1534.wav
(384000, 2) 2358.wav
(176400, 2) 7966.wav
(125167, 2) 4139.wav
(192000, 2) 1393.wav
(176400, 1) 6211.wav
(176400, 2) 6600.wav
(58441, 2) 7224.wav
(192000, 2) 2980.wav
(83349, 2) 8499.wav
(24935, 2) 2201.wav
(192000, 2) 4919.wav
(94662, 2) 4406.wav
(124800, 2) 607.wav
(176400, 2) 7379.wav
(192000, 2) 7027.wav
(192000, 1) 3369.wav
(36126, 2) 3323.wav
(384000, 2) 2983.wav
(176400, 2) 1651.wav
(176400, 2) 7278.wav
(192000, 2) 3446.wav
(176400, 2) 8314.wav
(192000, 2) 149.wav
(66720, 2) 2077.wav
(176400, 2) 2711.wav
(384000, 1) 4835.wav
(176400, 2) 5037.wav
(192000, 2) 3910.wav
(176400, 2) 1889.wav
(176400, 2) 2300.wav
(192000, 2) 5863.wav
(91974,

(176400, 2) 3747.wav
(176400, 2) 7251.wav
(176400, 1) 3154.wav
(176400, 2) 2070.wav
(176400, 2) 2049.wav
(26879, 2) 3492.wav
(44494, 2) 3236.wav
(192000, 2) 3632.wav
(176400, 2) 11.wav
(192000, 2) 8604.wav
(176400, 2) 5336.wav
(176400, 2) 4401.wav
(176400, 2) 3357.wav
(192000, 2) 5243.wav
(176400, 2) 851.wav
(176400, 1) 6629.wav
(51120, 2) 349.wav
(55018, 1) 1805.wav
(36119, 2) 7108.wav
(176400, 2) 5320.wav
(176400, 2) 3522.wav
(176400, 2) 1579.wav
(176400, 2) 4438.wav
(176400, 2) 1025.wav
(192000, 2) 3751.wav
(176400, 2) 1335.wav
(176400, 2) 5915.wav
(53802, 2) 6570.wav
(176400, 2) 953.wav
(176400, 2) 6992.wav
(176400, 2) 3745.wav
(384000, 2) 7744.wav
(176400, 2) 995.wav
(192000, 2) 4367.wav
(176400, 2) 6986.wav
(192000, 2) 8610.wav
(13451, 2) 7870.wav
(192000, 2) 8477.wav
(26901, 2) 3626.wav
(176400, 2) 5092.wav
(35277, 1) 2538.wav
(67768, 2) 7523.wav
(176400, 2) 5257.wav
(384000, 2) 1152.wav
(88200, 2) 5433.wav
(192000, 2) 7022.wav
(176400, 2) 5901.wav
(122309, 2) 5949.wav
(192000, 

...

(6720, 2) 7260.wav
(176400, 2) 385.wav
(176400, 2) 6118.wav
(176400, 2) 1773.wav
(192000, 2) 4531.wav
(192000, 2) 2318.wav
(176400, 2) 5571.wav
(37307, 2) 5515.wav
(192000, 2) 485.wav
(384000, 2) 2140.wav
(192000, 2) 4684.wav
(176400, 2) 5477.wav
(192000, 1) 1924.wav
(31752, 2) 8653.wav
(48960, 2) 7560.wav
(192000, 2) 6326.wav
(176400, 2) 4069.wav
(153600, 2) 268.wav
(192000, 2) 4769.wav
(384000, 2) 3855.wav
(192000, 2) 3706.wav
(192000, 1) 1449.wav
(176400, 2) 6332.wav
(176400, 2) 3006.wav
(192000, 2) 2593.wav
(176400, 2) 3762.wav
(384000, 2) 2418.wav
(176400, 2) 3760.wav
(192000, 2) 4525.wav
(384000, 2) 2587.wav
(176400, 2) 3004.wav
(176400, 2) 1105.wav
(176400, 2) 6618.wav
(176400, 2) 2154.wav
(176400, 2) 6939.wav
(176400, 2) 2632.wav
(384000, 2) 7506.wav
(384000, 2) 1613.wav
(176400, 2) 491.wav
(55192, 2) 7399.wav
(192000, 2) 8479.wav
(176400, 2) 8635.wav
(176400, 2) 7574.wav
(176400, 1) 6087.wav
(176400, 2) 7262.wav
(384000, 2) 4690.wav
(176400, 2) 7992.wav
(80658, 2) 8621.wav
(19

(186574, 2) 4190.wav
(192000, 2) 5980.wav
(51156, 2) 889.wav
(176400, 2) 6509.wav
(176400, 2) 651.wav
(176400, 2) 4446.wav
(176400, 2) 3471.wav
(176400, 2) 6537.wav
(192000, 2) 2800.wav
(26301, 2) 3317.wav
(72000, 2) 2009.wav
(96494, 2) 7629.wav
(192000, 2) 1500.wav
(176400, 2) 8493.wav
(176400, 2) 7171.wav
(176400, 2) 645.wav
(176400, 2) 4450.wav
(192000, 2) 7978.wav
(176400, 2) 5996.wav
(93933, 1) 123.wav
(192000, 2) 3288.wav
(192000, 2) 4578.wav
(192000, 2) 6247.wav
(51102, 2) 1400.wav
(176400, 2) 2196.wav
(176400, 2) 6409.wav
(384000, 2) 1528.wav
(176400, 2) 7159.wav
(192000, 2) 2828.wav
(176400, 2) 1366.wav
(176400, 2) 4281.wav
(176400, 2) 6521.wav
(192000, 2) 8424.wav
(176400, 2) 6480.wav
(122157, 2) 5028.wav
(192000, 2) 5100.wav
(34560, 1) 2747.wav
(147294, 2) 3459.wav
(58362, 2) 8485.wav
(192000, 2) 135.wav
(192000, 2) 42.wav
(176400, 2) 916.wav
(176400, 2) 121.wav
(110400, 2) 1649.wav
(176400, 2) 1891.wav
(138048, 2) 8518.wav
(176400, 2) 8491.wav
(176400, 2) 6132.wav
(176400, 

(176400, 2) 7942.wav
(384000, 2) 5403.wav
(176400, 2) 6460.wav
(176400, 2) 7275.wav
(384000, 2) 4436.wav
(192000, 2) 4241.wav
(384000, 2) 4254.wav
(192000, 2) 4137.wav
(176400, 2) 6484.wav
(44100, 1) 843.wav
(176400, 2) 3013.wav
(176400, 2) 2378.wav
(176400, 2) 4277.wav
(30265, 2) 5639.wav
(53361, 1) 2047.wav
(176400, 2) 2721.wav
(176400, 2) 4293.wav
(176400, 2) 3658.wav
(176400, 2) 8446.wav
(176400, 2) 3518.wav
(176400, 2) 6737.wav
(192000, 1) 5476.wav
(176400, 2) 2156.wav
(176400, 2) 5417.wav
(176400, 2) 5833.wav
(192000, 2) 2319.wav
(7441, 2) 3880.wav
(142002, 2) 5211.wav
(176400, 1) 4350.wav
(176400, 2) 1058.wav
(176400, 2) 848.wav
(384000, 2) 5169.wav
(14004, 2) 964.wav
(176400, 2) 6547.wav
(192000, 1) 3248.wav
(142410, 2) 3066.wav
(192000, 1) 2184.wav
(176400, 2) 3660.wav
(192000, 2) 8152.wav
(192000, 2) 4527.wav
(176400, 2) 2160.wav
(192000, 2) 7775.wav
(176400, 2) 7429.wav
(176400, 2) 119.wav
(66240, 2) 1228.wav
(192000, 2) 7507.wav
(192000, 2) 1077.wav
(176400, 2) 7881.wav
(17

(192000, 2) 7613.wav
(17561, 2) 4986.wav
(176400, 2) 1626.wav
(176400, 2) 3703.wav
(176400, 2) 1704.wav
(176400, 2) 7200.wav
(176400, 2) 8520.wav
(96000, 2) 4045.wav
(176400, 2) 7836.wav
(176400, 2) 905.wav
(88200, 1) 5414.wav
(176400, 2) 5562.wav
(176400, 2) 6149.wav
(192000, 2) 6652.wav
(384000, 2) 8130.wav
(176400, 2) 911.wav
(176400, 2) 8133.wav
(130560, 2) 1276.wav
(176400, 2) 7607.wav
(176400, 2) 7257.wav
(176400, 2) 5947.wav
(192000, 2) 7558.wav
(176400, 2) 3675.wav
(192000, 2) 7203.wav
(12171, 2) 7566.wav
(176400, 2) 6508.wav
(192000, 2) 3700.wav
(176400, 2) 5572.wav
(22919, 2) 2754.wav
(176400, 2) 4481.wav
(176400, 2) 4443.wav
(176400, 2) 7531.wav
(176400, 2) 1129.wav
(176400, 2) 8534.wav
(176400, 2) 1672.wav
(117810, 2) 1062.wav
(176400, 2) 6296.wav
(176400, 2) 8720.wav
(192000, 2) 1267.wav
(176400, 2) 5371.wav
(176400, 2) 3461.wav
(192000, 2) 5214.wav
(176400, 2) 5985.wav
(176400, 2) 2420.wav
(176400, 2) 2449.wav
(176400, 2) 1076.wav
(176400, 2) 7170.wav
(192000, 2) 7217.wav

(176400, 2) 6446.wav
(192000, 2) 4290.wav
(28320, 2) 1840.wav
(33516, 2) 6691.wav
(192000, 2) 4086.wav
(46761, 1) 8440.wav
(176400, 2) 7612.wav
(192000, 1) 1843.wav
(192000, 2) 7980.wav
(192000, 2) 5398.wav
(176400, 2) 8482.wav
(176400, 2) 4913.wav
(192000, 2) 3925.wav
(192000, 2) 5005.wav
(176400, 2) 1302.wav
(41895, 2) 327.wav
(176400, 2) 1329.wav
(192000, 2) 455.wav
(192000, 2) 654.wav
(88200, 1) 1854.wav
(192000, 1) 3474.wav
(192000, 1) 3515.wav
(176400, 2) 318.wav
(192000, 2) 333.wav
(176400, 2) 6083.wav
(131520, 2) 7969.wav
(176400, 2) 2620.wav
(192000, 2) 3273.wav
(192000, 2) 2811.wav
(176400, 2) 3258.wav
(176400, 2) 330.wav
(176400, 2) 4907.wav
(176400, 2) 278.wav
(14112, 2) 5818.wav
(64000, 1) 132.wav
(176400, 2) 7015.wav
(384000, 1) 2805.wav
(48069, 2) 130.wav
(192000, 2) 4904.wav
(192000, 2) 2813.wav
(176400, 2) 8496.wav
(192000, 2) 2345.wav
(176400, 2) 4534.wav
(176400, 2) 3851.wav
(63504, 2) 456.wav
(176400, 1) 6875.wav
(176400, 2) 898.wav
(176400, 2) 2597.wav
(176400, 2) 

In [20]:
noverlap<nperseg

True

In [21]:
noverlap

583

In [22]:
nperseg

598